In [1]:
import numpy as np
import warnings
import matplotlib.pyplot as plt
import folium

In [2]:
def map(x_min, x_max, y_min, y_max):
    interactive_map = folium.Map(
        location=((y_min+y_max)/2, (x_min+x_max)/2),
        zoom_start=min(int((np.log(1000 / (x_max-x_min)) / np.log(2))), int((np.log(1000 / (y_max-y_min)) / np.log(2)))),
        tiles="http://mt1.google.com/vt/lyrs=y&z={z}&x={x}&y={y}",
        attr="Google",
        prefer_canvas=True,
    )
    folium.TileLayer('openstreetmap', show=False).add_to(interactive_map)
    folium.TileLayer(
        tiles = 'https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}',
        attr = 'Esri',
        name = 'Esri Satellite').add_to(interactive_map)
    return interactive_map

In [5]:
#
# ANOMALIES
#
# NOTE: to obtain these predictions, need to run inference.py with save_npy=True
anoms = np.load("/data_2/scratch/dbrueggemann/output/anomalies_inf_exp.npy")  # N x 73 x H x W
coords = np.load("/data_2/scratch/dbrueggemann/output/coords_inf_exp.npy")  # N x 4

interactive_map = map(coords[0, 0], coords[0, 1], coords[0, 2], coords[0, 3])

# set the anomaly threshold
thresh = 1.5

anoms_per_month = np.array_split(anoms, 12, axis=1)
with warnings.catch_warnings():  # to ignore `mean of empty slice` warning
    warnings.simplefilter("ignore", category=RuntimeWarning)
    anoms_per_month = np.stack([np.nanmedian(el, axis=1) for el in anoms_per_month], axis=0)  # 12 x N x H x W

anoms_discrete = anoms_per_month.copy()
anoms_discrete[anoms_per_month >= thresh] = 1
anoms_discrete[anoms_per_month <= -thresh] = 0
anoms_discrete[(anoms_per_month > -thresh) & (anoms_per_month < thresh)] = 0.5

months = [
    "Jan", "Feb", "Mar", "Apr", "May", "Jun", "Jul", "Aug", "Sep", "Oct", "Nov", "Dec"
]

for j, (m, m_anoms) in enumerate(zip(months, anoms_discrete)):
    new_layer = folium.FeatureGroup(name=m).add_to(interactive_map)
    for i, (p, c) in enumerate(zip(m_anoms, coords)):
        bounds = [[float(c[2]), float(c[0])], [float(c[3]), float(c[1])]]
        # high values are red, low values are blue
        cm = plt.get_cmap('coolwarm')
        mask = np.isnan(p) | (p == 0.5)
        p_im = np.nan_to_num(p)
        colored_image = cm(p_im)
        # colored_image[:, :, 3] = np.abs(p_im * 2 - 1)
        colored_image[mask, 3] = 0
        folium.raster_layers.ImageOverlay(
            image=colored_image,
            bounds=bounds,
            mercator_project=True,
            opacity=0.8,
        ).add_to(new_layer)

        # if i > 200: break

folium.map.LayerControl(position='topright', collapsed=False, autoZIndex=True).add_to(interactive_map)

interactive_map.save("/data_2/scratch/dbrueggemann/output/anomalies_map.html")

In [11]:
# optionally visualize map in notebook:
interactive_map

In [3]:
#
# FOREST MASK
#
# NOTE: to obtain these predictions, need to run inference.py with save_npy=True
forest_mask = np.load("/data_2/scratch/dbrueggemann/output/forest_mask_inf_exp.npy")  # N x H x W
coords = np.load("/data_2/scratch/dbrueggemann/output/coords_inf_exp.npy")  # N x 4

interactive_map = map(coords[0, 0], coords[0, 1], coords[0, 2], coords[0, 3])

mask = forest_mask.astype(int)

forest_layers = folium.FeatureGroup(name='Forest Mask').add_to(interactive_map)
for i, (p, c) in enumerate(zip(mask, coords)):
    bounds = [[float(c[2]), float(c[0])], [float(c[3]), float(c[1])]]
    # forest mask is red
    cm = plt.get_cmap('Reds')
    nan_mask = (p == 0)
    colored_image = cm(p)
    colored_image = np.stack([np.ones_like(p), np.zeros_like(p), np.zeros_like(p), np.ones_like(p)], axis=2)
    colored_image[nan_mask, 3] = 0
    folium.raster_layers.ImageOverlay(
        image=colored_image,
        bounds=bounds,
        mercator_project=True,
        opacity=0.7,
    ).add_to(forest_layers)

folium.map.LayerControl(position='topright', collapsed=False, autoZIndex=True).add_to(interactive_map)

interactive_map.save("/data_2/scratch/dbrueggemann/output/forest_mask_map.html")

In [4]:
#
# PARAMETERS
#
# NOTE: to obtain these predictions, need to run inference.py with save_npy=True
params = np.load("/data_2/scratch/dbrueggemann/output/parameters_inf_exp.npy")  # N x H x W x 6
coords = np.load("/data_2/scratch/dbrueggemann/output/coords_inf_exp.npy")  # N x 4

interactive_map = map(coords[0, 0], coords[0, 1], coords[0, 2], coords[0, 3])

# SOS
im_params = params[:, :, :, 0]
# scale to [0,1]
mini = np.percentile(im_params[~np.isnan(im_params)], 2)
maxi = np.percentile(im_params[~np.isnan(im_params)], 98)
im_params = (im_params - mini) / (maxi - mini)
im_params[im_params < 0] = 0
im_params[im_params > 1] = 1

sos_layers = folium.FeatureGroup(name='SOS').add_to(interactive_map)
for i, (p, c) in enumerate(zip(im_params, coords)):
    bounds = [[float(c[2]), float(c[0])], [float(c[3]), float(c[1])]]
    # high values are red, low values are blue (middle are white)
    cm = plt.get_cmap('coolwarm')
    nan_mask = np.isnan(p)
    p_im = np.nan_to_num(p)
    colored_image = cm(p_im)
    colored_image[nan_mask, 3] = 0
    folium.raster_layers.ImageOverlay(
        image=colored_image,
        bounds=bounds,
        mercator_project=True,
        opacity=0.8,
    ).add_to(sos_layers)

# EOS
im_params = params[:, :, :, 2] + np.logaddexp(0, params[:, :, :, 3])
# scale to [0,1]
mini = np.percentile(im_params[~np.isnan(im_params)], 2)
maxi = np.percentile(im_params[~np.isnan(im_params)], 98)
im_params = (im_params - mini) / (maxi - mini)
im_params[im_params < 0] = 0
im_params[im_params > 1] = 1

eos_layers = folium.FeatureGroup(name='EOS').add_to(interactive_map)
for i, (p, c) in enumerate(zip(im_params, coords)):
    bounds = [[float(c[2]), float(c[0])], [float(c[3]), float(c[1])]]
    # high values are red, low values are blue (middle are white)
    cm = plt.get_cmap('coolwarm')
    nan_mask = np.isnan(p)
    p_im = np.nan_to_num(p)
    colored_image = cm(p_im)
    colored_image[nan_mask, 3] = 0
    folium.raster_layers.ImageOverlay(
        image=colored_image,
        bounds=bounds,
        mercator_project=True,
        opacity=0.8,
    ).add_to(eos_layers)

# summer NDVI
im_params = params[:, :, :, 4]
# scale to [0,1]
mini = np.percentile(im_params[~np.isnan(im_params)], 2)
maxi = np.percentile(im_params[~np.isnan(im_params)], 98)
im_params = (im_params - mini) / (maxi - mini)
im_params[im_params < 0] = 0
im_params[im_params > 1] = 1

sndvi_layers = folium.FeatureGroup(name='summer NDVI').add_to(interactive_map)
for i, (p, c) in enumerate(zip(im_params, coords)):
    bounds = [[float(c[2]), float(c[0])], [float(c[3]), float(c[1])]]
    # high values are red, low values are blue (middle are white)
    cm = plt.get_cmap('coolwarm')
    nan_mask = np.isnan(p)
    p_im = np.nan_to_num(p)
    colored_image = cm(p_im)
    colored_image[nan_mask, 3] = 0
    folium.raster_layers.ImageOverlay(
        image=colored_image,
        bounds=bounds,
        mercator_project=True,
        opacity=0.8,
    ).add_to(sndvi_layers)

# winter NDVI
im_params = params[:, :, :, 5]
# scale to [0,1]
mini = np.percentile(im_params[~np.isnan(im_params)], 2)
maxi = np.percentile(im_params[~np.isnan(im_params)], 98)
im_params = (im_params - mini) / (maxi - mini)
im_params[im_params < 0] = 0
im_params[im_params > 1] = 1

wndvi_layers = folium.FeatureGroup(name='winter NDVI').add_to(interactive_map)
for i, (p, c) in enumerate(zip(im_params, coords)):
    bounds = [[float(c[2]), float(c[0])], [float(c[3]), float(c[1])]]
    # high values are red, low values are blue (middle are white)
    cm = plt.get_cmap('coolwarm')
    nan_mask = np.isnan(p)
    p_im = np.nan_to_num(p)
    colored_image = cm(p_im)
    colored_image[nan_mask, 3] = 0
    folium.raster_layers.ImageOverlay(
        image=colored_image,
        bounds=bounds,
        mercator_project=True,
        opacity=0.8,
    ).add_to(wndvi_layers)

folium.map.LayerControl(position='topright', collapsed=False, autoZIndex=True).add_to(interactive_map)

interactive_map.save("/data_2/scratch/dbrueggemann/output/parameters_map.html")

/tmp/ipykernel_1243297/3983400437.py:36: RuntimeWarning: invalid value encountered in logaddexp
  im_params = params[:, :, :, 2] + np.logaddexp(0, params[:, :, :, 3])
